<summary>
    <font size="5" color="gray"><b> CONVENIO CENACE-UNISON 2023-2025
 </b></font>
</summary><br>
<summary>
    <font size="4" color="orange"><b> DISEÑO E IMPLEMENTACIÓN DE UNA METODOLOGÍA PARA PRONÓSTICO DE DEMANDA DE ENERGÍA A CORTO PLAZO CON MANEJO DE DATOS BAJO INCERTIDUMBRE
 </b></font>
</summary>

---

<summary>
    <font size="6" color="gray"><b> Libreta JNB2: Entrenamiento 
 </b></font>
</summary>
<summary>
    <font size="5" color="gray"><b> Modelado parte 2
 </b></font>
</summary>

--- 

<summary>
    <font size="4" color="lightblue"><b> MODELO ENCODER-DECODER CON SEÑALES DE DÍAS FESTIVOS Y CLIMA EN ADELANTO
 </b></font>
</summary>

---

    
OLIVIA CAROLINA GUTÚ OCAMPO

REYNA YANET HERNÁNDEZ MADA

JULIO WAISSMAN VILANOVA (Responsable del proyecto)


---


<summary>
    <font size="4" color="orange"><b> INPUTS
 </b></font>
</summary>

   * **'./internal_inputs/TENSORS/X_encoder_train.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_train.npy'** 
   * **'./internal_inputs/TENSORS/y_train.npy'** 
   * **'./internal_inputs/TENSORS/X_encoder_val.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_val.npy'** 
   * **'./internal_inputs/TENSORS/y_val.npy'** 
   * **'./internal_inputs/TENSORS/X_encoder_test.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_test.npy'** 
   * **'./internal_inputs/TENSORS/y_test.npy'** 

--- 
<summary>
    <font size="4" color="orange"><b> CHECK POINTS
 </b></font>
</summary>

   * Verificación de curvas de aprendizaje.
    
   * Verificación de performace (MAPE) sobre el conjunto de prueba.

---
<summary>
    <font size="4" color="orange"><b> OUTPUTS
 </b></font>
</summary>

Se exportan finalmente, el pronóstico sobre el tensor de datos de prueba (antes del escalamiento final), las curvas de aprendizaje y el modelo.
    
   *  **'./outputs/MODEL/y_estimado_antes_minmax.npy'**
   *  **'./outputs/MODEL/training_loss.npy'**
   *  **'./outputs/MODEL/val_loss.npy'**
   *  **'./outputs/MODEL/modelo.h5'**


## 1.   Importación de librerías 

In [1]:
# librerías básicas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# librerías de normalización
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# Tensorflow y keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Auxiliar libraries
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='notebook'
import pickle as pk
import scipy
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

## 2.  Importando tensores de entrada

In [2]:
# Importación de  tensors
X_encoder_train = np.load('./internal_inputs/TENSORS/X_encoder_train.npy')
X_decoder_train = np.load('./internal_inputs/TENSORS/X_decoder_train.npy')
y_train = np.load('./internal_inputs/TENSORS/y_train.npy')
X_encoder_val = np.load('./internal_inputs/TENSORS/X_encoder_val.npy')
X_decoder_val = np.load('./internal_inputs/TENSORS/X_decoder_val.npy')
y_val = np.load('./internal_inputs/TENSORS/y_val.npy')
X_encoder_test = np.load('./internal_inputs/TENSORS/X_encoder_test.npy')
X_decoder_test = np.load('./internal_inputs/TENSORS/X_decoder_test.npy') 
y_test = np.load('./internal_inputs/TENSORS/y_test.npy')

## 3.   Ajuste de modelo

In [3]:
## Setting the features
all_features = ['Energy_Demand', 
                'Day', 
                'Hour', 
                'Month', 
                'PC1_Weather',
                'PC2_Weather',
                'Monday_Holiday', 
                'Tuesday_Aft_Hol', 
                'Easter_week',
                'May_1s', 
                'May_10t', 
                'Sept_16', 
                'Nov_2nd', 
                'Before_Christmas_NY',
                'Christmas_NY', 
                'After_Christmas_NY'
               ]

In [4]:
#Ajuste de hiperparámetros
n_pasado = (24 * 7) + 12
n_futuro = 36 
m = len(all_features)

In [8]:
# Ajuste encoder

encoder_inputs = layers.Input(shape = (n_pasado, m), name = 'encoder_inputs')

encoder_l1 = layers.LSTM(100, return_state = True) # 100: dimension of hidden states
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

# Ajuste decoder
decoder_inputs = layers.Input(
    shape = (n_futuro, 12), name = 'decoder_inputs') # Future given in advance

decoder_l1 = layers.LSTM(100, return_sequences=True) # 100: dimension of hidden states
decoder_l1_output = decoder_l1(decoder_inputs, initial_state = encoder_states1)

decoder_l2 = layers.TimeDistributed(layers.Dense(1)) # just one dense layer
decoder_outputs = decoder_l2(decoder_l1_output)

2023-09-08 18:46:47.411318: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
modelo = tf.keras.models.Model(
    inputs = [encoder_inputs, decoder_inputs], outputs = decoder_outputs)
modelo.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 180, 16)]    0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 36, 12)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 100),        46800       ['encoder_inputs[0][0]']         
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                              

## 4. Entrenamiento del modelo

In [ ]:
#Learning rate es menor en cada epoch
reduce_lr = keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x) 

## Se guarda el mejor modelo, de acuerdo a "val_loss"
path_checkpoint = './outputs/MODEL/model_checkpoint.h5'
modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor = 'val_loss',
    filepath = path_checkpoint,
    verbose = 1,
    save_weights_only = True,
    save_best_only = True,
)

## Para evitar overfitting, se detiene el entrenamiento despues
# # de epochs sin mejora, de acuerdo a "val_loss"
es_callback = keras.callbacks.EarlyStopping(
    monitor = 'val_loss', 
    min_delta = 0, 
    patience = 5 
    
)
modelo.compile(
    optimizer = keras.optimizers.Adam(), 
    loss = 'mae', 
)

history = modelo.fit(
    [X_encoder_train,X_decoder_train], y_train,
    epochs = 25, 
    validation_data = ([X_encoder_val, X_decoder_val], y_val),
    batch_size = 32,
    callbacks = [reduce_lr, es_callback]
) 

if es_callback.stopped_epoch < 25:
    print("Model stopped training early at epoch", es_callback.stopped_epoch + 1)
else:
    print("Model completed training for all 25 epochs")

## 5.   Check point: verificar learning curves

In [ ]:
# Model Loss Chart
fig = plt.figure()
plt.figure(figsize=(36, 13))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

In [11]:
# Exportar loss values durante el entrenamiento por  epoch
np.save('./outputs/MODEL/training_loss.npy', history.history['loss'])
np.save('./outputs/MODEL/val_loss.npy', history.history['val_loss'])

## 6.   Check point: verificar performance  set de prueba

In [12]:
y_est = modelo.predict([X_encoder_test, X_decoder_test])

In [13]:
#Exportar el tensor de pronóstico antes de  transformar al  scaler original
np.save('./outputs/MODEL/y_estimado_antes_minmax.npy', y_est)

In [14]:
scalers = pk.load(open("./internal_inputs/MINMAX/scalers.pkl",'rb'))
scaler = scalers['Energy_Demand']

In [ ]:
n = y_est.shape[0]
forecast_mape = []
for i in range(n):
    y_test_original = scaler.inverse_transform(y_test[i-1])
    y_est_final = scaler.inverse_transform(y_est[i-1])
    mape = mean_absolute_percentage_error(y_test_original,y_est_final) * 100
    forecast_mape.append(mape)
plt.hist(forecast_mape)

## 7.   Guardado de modelo

In [11]:
modelo.save('./outputs/MODEL/modelo_3.h5')

<summary>
    <font size="4" color="gray"> Maestría en Ciencia de Datos | Universidad de Sonora </font>
</summary>
<font size="1" color="gray"> Blvd. Luis Encinas y Rosales s/n Col. Centro. Edificio 3K1 planta baja C.P. 83000, Hermosillo, Sonora, México </font>
<font size="1" color="gray"> mcd@unison.mx </font>
<font size="1" color="gray"> Tel: +52 (662) 259 2155  </font>